In [3]:
from tqdm import tqdm
import re 

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader, DirectoryLoader
from src.utils import reformat_text
from src.vectorstore import jsonize_document

import pandas as pd
import json

import random
import string
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

from pymilvus import (
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection, AnnSearchRequest, RRFRanker, connections
)

from pymilvus.model.hybrid import BGEM3EmbeddingFunction

In [5]:
connections.connect("default", host="localhost", port="19530")

fields = [
    FieldSchema(name="pk", dtype=DataType.VARCHAR,
                is_primary=True, auto_id=True, max_length=100),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=3000),
    FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=1024),
]

schema= CollectionSchema(fields, "")

col = Collection("sparse_dense_demo_huberman", schema)

sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
dense_index = {"index_type": "FLAT", "metric_type": "COSINE"}

col.create_index("sparse_vector", sparse_index)
col.create_index("dense_vector", dense_index)

Status(code=0, message=)

In [6]:
loader = DirectoryLoader("docs/youtube_test/", glob="*.txt", show_progress=True)
docs = loader.load()

for i, doc in enumerate(docs):
    doc.page_content = reformat_text(doc.page_content)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=100,
    length_function=len,
)

splitted_docs = text_splitter.split_documents(docs)

100%|██████████| 1/1 [00:12<00:00, 12.62s/it]


In [7]:
ef = BGEM3EmbeddingFunction(use_fp16=False, device="cpu")

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 167996.15it/s]


In [10]:
for doc in tqdm(splitted_docs, desc="Processing texts"):
    doc = doc.page_content
    print("Embedding...")
    doc_embeddings = ef([doc])
    print("Embedding finished.")
    entity = [[doc], [doc_embeddings["sparse"]], [doc_embeddings["dense"]]]
    print("Inserting...")
    col.insert([entity])
    col.flush()
    print("Finished inserting.")

Processing texts:   0%|          | 0/46 [00:00<?, ?it/s]

Embedding...


Processing texts:   0%|          | 0/46 [00:02<?, ?it/s]

Embedding finished.
Inserting...


DataNotMatchException: <DataNotMatchException: (code=1, message=The data don't match with schema fields, expect 3 list, got 1)>